In [ ]:
%config IPCompleter.greedy=True
import numpy as np
import pandas as pd
import tensorflow as tf
from typing import List
from jpype import JClass, JString, getDefaultJVMPath, shutdownJVM, startJVM, java
import pandas as pd
from os.path import join
from sklearn.utils import shuffle
from pandas import ExcelWriter
import re
from nltk.corpus import stopwords
import emoji
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence,text
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report,f1_score,recall_score
import seaborn as sns
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,Flatten
from keras.layers import  Embedding, SimpleRNN, LSTM,Masking,Bidirectional
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.models import model_from_json,model_from_yaml,load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence,text
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras import metrics, regularizers
from keras.optimizers import RMSprop
from keras import backend as K
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE

In [ ]:
def remove_emoji(string):
    return emoji.get_emoji_regexp().sub(u' ', string)

In [ ]:
ZEMBEREK_PATH = "./bin/zemberek-full.jar"

startJVM(getDefaultJVMPath(), '-ea', '-Djava.class.path=%s' % (ZEMBEREK_PATH))

In [ ]:
dosya = ['Trendyol.xlsx','HepsiBurada.xlsx','GittiGidiyor.xlsx']

for item in dosya:
    data = pd.read_excel(item)

    # tüm harfler küçüğe dönüştülür
    data = data.apply(lambda x: x.astype(str).str.lower())

    # Regex işlemleri
    for i in range(len(data)):

        # https://t.co/osUgGrOJSz kelimeleri silme
        output = re.sub(r"http[s]?:(?:[a-zA-Z]|[0-9]|[$-_@.& +]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", "",
                        data['tweet'][i])

        # @kullanıcı adlarını ve etiketleri silme
        output = re.sub(r"\@\w*\b", "", output)
        output = re.sub(r"\#\w*\b", "", output)
        # Rakamları temizleme
        output = re.sub(r"\b\d+", "", output)

        # 3 karakterden az olanları silme
        output = re.sub(r"\W*\b\w{1,2}\b", "", output)
        
        #emoji kaldırma
        output=remove_emoji(output)

        data['tweet'][i] = output

    ## Karakterleri temizleme
    data['tweet'] = data['tweet'].str.findall('\w{2,}').str.join(' ')

    ## Kelime düzeltme işlemi
    TurkishMorphology: JClass = JClass('zemberek.morphology.TurkishMorphology')
    TurkishSentenceNormalizer: JClass = JClass(
        'zemberek.normalization.TurkishSentenceNormalizer'
    )
    Paths: JClass = JClass('java.nio.file.Paths')

    normalizer = TurkishSentenceNormalizer(
        TurkishMorphology.createWithDefaults(),
        Paths.get(join('data', 'normalization')),
        Paths.get(join('data', 'lm', 'lm.2gram.slm'))
    )

    for num in range(len(data.tweet)):
        # print((
        #    f'\nNormal      : {data.tweet[num]}'
        #    f'\nDuzenlenmiş : {normalizer.normalize(JString(data.tweet[num]))}'
        #
        #        ))
        data.xs(num)['tweet'] = normalizer.normalize(JString(data.tweet[num]))

    ## Kelime köklerini bulma ve anlamsızlar UNK yazar
    
    TurkishMorphology = JClass('zemberek.morphology.TurkishMorphology')
    morphology = TurkishMorphology.createWithDefaults()
    
    for num in range(len(data.tweet)):
       # print("Orjinal tweet --->", data.tweet[num])
        analysis: java.util.ArrayList = (morphology.analyzeAndDisambiguate(data.tweet[num]).bestAnalysis())
        pos: List[str] = []
        for i, analysis in enumerate(analysis, start=1):
            f'\nAnalysis {i}: {analysis}', f'\nPrimary POS {i}: {analysis.getPos()}' f'\nPrimary POS (Short Form) {i}: {analysis.getPos().shortForm}'
            pos.append(
                f'{str(analysis.getLemmas()[0])}'
            )
        data.xs(num)['tweet'] = " ".join(pos)
       # print(f'Islem Gormus  ---> {" ".join(pos)}',"\n")
    
    data['tweet']=data['tweet'].str.replace("UNK"," ")
    #data['tweet']=data['tweet'].str.findall('\w{UNK}').str.join(' ')
    

    stop_words = stopwords.words('turkish')
    with open('turkce-stop-words.txt', encoding='utf-8') as file:
        stw = file.read()
    stw = stw.split()
    stw = [s.lower() for s in stw]
    stop_words += stw

    data['tweet'] = data['tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

    data = shuffle(data)

    ad = 'temizlendi' + item
    writer = ExcelWriter(ad)
    data.to_excel(writer, item)
    writer.save()

shutdownJVM()

In [ ]:
filename="hepsi"
data = pd.read_excel('./'+filename+'.xlsx')
data.dropna( inplace = True)
duplicateDFRow = data[data.duplicated(['tweet'])]
data.drop_duplicates(subset ="tweet",keep = False, inplace = True)
data = data.reset_index(drop=True)
tags = data["tags"]
texts = data["tweet"]
data = shuffle(data)
data.head()

In [ ]:
### Tagsleri 0 ve 1 yapma (Kategorik verileri nümerik hale getirme)
le = LabelEncoder()
tags = le.fit_transform(tags)

In [ ]:
# Kelime numalarandırmada en büyük sayı
num_max = 10000
## Kelime numaralandırma işlemi
tok = Tokenizer(num_words=num_max)
tok.fit_on_texts(texts)

In [ ]:
# En uzun tweet ve farklı kelime sayısı
max_len = 15
vocab_size=10000

In [ ]:
#Kelime numaralandırma örneği
print(texts[10])
for item in texts[10].split():    
    print(tok.word_index[item])

In [ ]:
## Tüm tweetleri numaralandırma
cnn_texts_seq = tok.texts_to_sequences(texts)

In [ ]:
# Numaralandırılmış tweetleri belirli bir uzunluğa getirme ve padding işlemi
cnn_texts_mat = tf.keras.preprocessing.sequence.pad_sequences(cnn_texts_seq,maxlen=max_len,padding='post')

# Örnek
print('***************************************************')
print(texts[40])
print(cnn_texts_mat[40])
print('***************************************************')

In [ ]:
dataset_trendyol=pd.read_excel("temizlendiTrendyol.xlsx")
h=dataset_trendyol['tags']=='negatif'
print("Negatif count" ,len(dataset_trendyol[h]))
h=dataset_trendyol['tags']=='pozitif'
print("Pozitif count" ,len(dataset_trendyol[h]))
dataset_trendyol.iloc[:,1] = le.fit_transform(dataset_trendyol.iloc[:,1])
X_train_t,X_test_t,y_train_t,y_test_t=train_test_split(cnn_texts_mat,tags,test_size=0.2)
dataset_trendyol

In [ ]:
dataset_hepsiburada=pd.read_excel("temizlendiHepsiBurada.xlsx")
h=dataset_hepsiburada['tags']=='negatif'
print("Negatif count" ,len(dataset_hepsiburada[h]))
h=dataset_hepsiburada['tags']=='pozitif'
print("Pozitif count" ,len(dataset_hepsiburada[h]))
dataset_hepsiburada.iloc[:,1] = le.fit_transform(dataset_hepsiburada.iloc[:,1])
X_train_h,X_test_h,y_train_h,y_test_h=train_test_split(cnn_texts_mat,tags,test_size=0.2)
dataset_hepsiburada

In [ ]:
dataset_gittigidiyor=pd.read_excel("temizlendiGittiGidiyor.xlsx")
h=dataset_gittigidiyor['tags']=='negatif'
print("Negatif count" ,len(dataset_gittigidiyor[h]))
h=dataset_gittigidiyor['tags']=='pozitif'
print("Pozitif count" ,len(dataset_gittigidiyor[h]))
dataset_gittigidiyor.iloc[:,1] = le.fit_transform(dataset_gittigidiyor.iloc[:,1])
X_train_g,X_test_g,y_train_g,y_test_g=train_test_split(cnn_texts_mat,tags,test_size=0.2)
dataset_gittigidiyor

In [ ]:
dataset_all=pd.read_excel("hepsi.xlsx")
h=dataset_all['tags']=='negatif'
print("Negatif count" ,len(dataset_all[h]))
h=dataset_all['tags']=='pozitif'
print("Pozitif count" ,len(dataset_all[h]))
dataset_all.iloc[:,1] = le.fit_transform(dataset_all.iloc[:,1])
X_train_a,X_test_a,y_train_a,y_test_a=train_test_split(cnn_texts_mat,tags,test_size=0.2)
dataset_all

In [ ]:
#LSTM - Trendyol
model=Sequential()
model.add(Embedding(vocab_size,max_len, trainable=True,input_length=max_len))
model.add(LSTM(32,activation='relu',kernel_regularizer=regularizers.l2(0.01),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(16,activation='relu',kernel_regularizer=regularizers.l2(0.01),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(4,activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer=RMSprop(lr=0.001),metrics=['accuracy'])
history_t_l=model.fit(X_train_t,y_train_t, epochs=10, batch_size=8,
                      verbose=1,validation_split=0.1)

In [ ]:
scores = model.evaluate(X_train_t,y_train_t,batch_size=1)
print("Training Accuracy: %.2f%%\n" % (scores[1]*100))
scores = model.evaluate(X_test_t,y_test_t,batch_size=1)
print("Testing Accuracy: %.2f%%\n" % (scores[1]*100))

In [ ]:
y_pred_test = model.predict(X_test_t)
y_pred=[]
for i in y_pred_test:
    if(i>=0.5):
        y_pred.append(1)
    else:
        y_pred.append(0)
cm = confusion_matrix(y_test_t,y_pred)
ax = sns.heatmap(cm, annot=True, xticklabels=["Pozitif",'Negatif'], yticklabels=['Pozitif','Negatif'],
                cbar=False,cmap='Blues')
ax.set_xlabel('Prediction')
ax.set_ylabel('Actual')
plt.show()

In [ ]:
print(classification_report(y_test_t, y_pred))

In [ ]:
#LSTM - HepsiBurada
model=Sequential()
model.add(Embedding(vocab_size,max_len, trainable=True,input_length=max_len))
model.add(LSTM(32,activation='relu',kernel_regularizer=regularizers.l2(0.01),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(16,activation='relu',kernel_regularizer=regularizers.l2(0.01),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(4,activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer=RMSprop(lr=0.001),metrics=['acc'])
history_h_l=model.fit(X_train_h,y_train_h, epochs=10, batch_size=8,
                      verbose=1,validation_split=0.1)

In [ ]:
scores = model.evaluate(X_train_h,y_train_h,batch_size=1)
print("Training Accuracy: %.2f%%\n" % (scores[1]*100))
scores = model.evaluate(X_test_h,y_test_h,batch_size=1)
print("Testing Accuracy: %.2f%%\n" % (scores[1]*100))

In [ ]:
y_pred_test = model.predict(X_test_h)
y_pred=[]
for i in y_pred_test:
    if(i>=0.5):
        y_pred.append(1)
    else:
        y_pred.append(0)
cm = confusion_matrix(y_test_h,y_pred)
ax = sns.heatmap(cm, annot=True, xticklabels=["Pozitif",'Negatif'], yticklabels=['Pozitif','Negatif'],
                cbar=False,cmap='Blues')
ax.set_xlabel('Prediction')
ax.set_ylabel('Actual')
plt.show()

In [ ]:
print(classification_report(y_test_h, y_pred))

In [ ]:
#LSTM - GittiGidiyor
model=Sequential()
model.add(Embedding(vocab_size,max_len, trainable=True,input_length=max_len))
model.add(LSTM(32,activation='relu',kernel_regularizer=regularizers.l2(0.01),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(16,activation='relu',kernel_regularizer=regularizers.l2(0.01),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(4,activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer=RMSprop(lr=0.001),metrics=['acc'])
history_g_l=model.fit(X_train_g,y_train_g, epochs=10, batch_size=8,
                      verbose=1,validation_split=0.1)

In [ ]:
scores = model.evaluate(X_train_g,y_train_g,batch_size=1)
print("Training Accuracy: %.2f%%\n" % (scores[1]*100))
scores = model.evaluate(X_test_g,y_test_g,batch_size=1)
print("Testing Accuracy: %.2f%%\n" % (scores[1]*100))

In [ ]:
y_pred_test = model.predict(X_test_g)
y_pred=[]
for i in y_pred_test:
    if(i>=0.5):
        y_pred.append(1)
    else:
        y_pred.append(0)
cm = confusion_matrix(y_test_g,y_pred)
ax = sns.heatmap(cm, annot=True, xticklabels=["Pozitif",'Negatif'], yticklabels=['Pozitif','Negatif'],
                cbar=False,cmap='Blues')
ax.set_xlabel('Prediction')
ax.set_ylabel('Actual')
plt.show()

In [ ]:
print(classification_report(y_test_g, y_pred))

In [ ]:
#Naive Bayes - Trendyol
classifier = GaussianNB()
history_nb_t = classifier.fit(X_train_t, y_train_t)
y_pred_nb = classifier.predict(X_test_t)
scores = accuracy_score(y_test_t, y_pred_nb)
print(scores)

In [ ]:
y_pred_test = classifier.predict(X_test_t)
y_pred=[]
for i in y_pred_test:
    if(i>=0.5):
        y_pred.append(1)
    else:
        y_pred.append(0)
cm = confusion_matrix(y_test_t,y_pred)
ax = sns.heatmap(cm, annot=True, xticklabels=["Pozitif",'Negatif'], yticklabels=['Pozitif','Negatif'],
                cbar=False,cmap='Blues')
ax.set_xlabel('Prediction')
ax.set_ylabel('Actual')
plt.show()

In [ ]:
print(classification_report(y_test_t, y_pred))

In [ ]:
#Naive Bayes - HepsiBurada
classifier = GaussianNB()
history_nb_t = classifier.fit(X_train_h, y_train_h)
y_pred_nb = classifier.predict(X_test_h)
scores = accuracy_score(y_test_h, y_pred_nb)
print(scores)

In [ ]:
y_pred_test = classifier.predict(X_test_h)
y_pred=[]
for i in y_pred_test:
    if(i>=0.5):
        y_pred.append(1)
    else:
        y_pred.append(0)
cm = confusion_matrix(y_test_h,y_pred)
ax = sns.heatmap(cm, annot=True, xticklabels=["Pozitif",'Negatif'], yticklabels=['Pozitif','Negatif'],
                cbar=False,cmap='Blues')
ax.set_xlabel('Prediction')
ax.set_ylabel('Actual')
plt.show()

In [ ]:
print(classification_report(y_test_h, y_pred))

In [ ]:
#Naive Bayes - GittiGidiyor
classifier = GaussianNB()
history_nb_t = classifier.fit(X_train_g, y_train_g)
y_pred_nb = classifier.predict(X_test_g)
scores = accuracy_score(y_test_g, y_pred_nb)
print(scores)

In [ ]:
y_pred_test = classifier.predict(X_test_g)
y_pred=[]
for i in y_pred_test:
    if(i>=0.5):
        y_pred.append(1)
    else:
        y_pred.append(0)
cm = confusion_matrix(y_test_g,y_pred)
ax = sns.heatmap(cm, annot=True, xticklabels=["Pozitif",'Negatif'], yticklabels=['Pozitif','Negatif'],
                cbar=False,cmap='Blues')
ax.set_xlabel('Prediction')
ax.set_ylabel('Actual')
plt.show()

In [ ]:
print(classification_report(y_test_g, y_pred))

In [ ]:
# Random Forest - Trendyol
rf = RandomForestClassifier(n_estimators=100, max_depth = 3, random_state=32)
rf.fit(X_train_t, y_train_t)
predictions = rf.predict(X_test_t)
score = round(accuracy_score(y_test_t, predictions), 3)
score

In [ ]:
y_pred_test = rf.predict(X_test_t)
y_pred=[]
for i in y_pred_test:
    if(i>=0.5):
        y_pred.append(1)
    else:
        y_pred.append(0)
cm = confusion_matrix(y_test_t,y_pred)
ax = sns.heatmap(cm, annot=True, xticklabels=["Pozitif",'Negatif'], yticklabels=['Pozitif','Negatif'],
                cbar=False,cmap='Blues')
ax.set_xlabel('Prediction')
ax.set_ylabel('Actual')
plt.show()

In [ ]:
print(classification_report(y_test_t, y_pred))

In [ ]:
# Random Forest - HepsiBurada
rf = RandomForestClassifier(n_estimators=100, max_depth = 3, random_state=32)
rf.fit(X_train_h, y_train_h)
predictions = rf.predict(X_test_h)
score = round(accuracy_score(y_test_h, predictions), 3)
score

In [ ]:
y_pred_test = rf.predict(X_test_h)
y_pred=[]
for i in y_pred_test:
    if(i>=0.5):
        y_pred.append(1)
    else:
        y_pred.append(0)
cm = confusion_matrix(y_test_h,y_pred)
ax = sns.heatmap(cm, annot=True, xticklabels=["Pozitif",'Negatif'], yticklabels=['Pozitif','Negatif'],
                cbar=False,cmap='Blues')
ax.set_xlabel('Prediction')
ax.set_ylabel('Actual')
plt.show()

In [ ]:
print(classification_report(y_test_h, y_pred))

In [ ]:
# Random Forest - GittiGidiyor
rf = RandomForestClassifier(n_estimators=100, max_depth = 3, random_state=32)
rf.fit(X_train_g, y_train_g)
predictions = rf.predict(X_test_g)
score = round(accuracy_score(y_test_g, predictions), 3)
score

In [ ]:
y_pred_test = rf.predict(X_test_g)
y_pred=[]
for i in y_pred_test:
    if(i>=0.5):
        y_pred.append(1)
    else:
        y_pred.append(0)
cm = confusion_matrix(y_test_g,y_pred)
ax = sns.heatmap(cm, annot=True, xticklabels=["Pozitif",'Negatif'], yticklabels=['Pozitif','Negatif'],
                cbar=False,cmap='Blues')
ax.set_xlabel('Prediction')
ax.set_ylabel('Actual')
plt.show()

In [ ]:
print(classification_report(y_test_g, y_pred))

In [ ]:
# Tüm veri seti - LSTM
model=Sequential()
model.add(Embedding(vocab_size,max_len, trainable=True,input_length=max_len))
model.add(LSTM(32,activation='relu',kernel_regularizer=regularizers.l2(0.01),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(16,activation='relu',kernel_regularizer=regularizers.l2(0.01),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(4,activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer=RMSprop(lr=0.001),metrics=['accuracy'])
history_a_l=model.fit(X_train_a,y_train_a, epochs=10, batch_size=8,
                      verbose=1,validation_split=0.1)

In [ ]:
scores = model.evaluate(X_train_a,y_train_a,batch_size=1)
print("Training Accuracy: %.2f%%\n" % (scores[1]*100))
scores = model.evaluate(X_test_a,y_test_a,batch_size=1)
print("Testing Accuracy: %.2f%%\n" % (scores[1]*100))

In [ ]:
y_pred_test = model.predict(X_test_a)
y_pred=[]
for i in y_pred_test:
    if(i>=0.5):
        y_pred.append(1)
    else:
        y_pred.append(0)
cm = confusion_matrix(y_test_a,y_pred)
ax = sns.heatmap(cm, annot=True, xticklabels=["Pozitif",'Negatif'], yticklabels=['Pozitif','Negatif'],
                cbar=False,cmap='Blues')
ax.set_xlabel('Prediction')
ax.set_ylabel('Actual')
plt.show()

In [ ]:
print(classification_report(y_test_a, y_pred))

In [ ]:
# Tüm veri seti - Naive Bayes 
classifier = GaussianNB()
history_nb_t = classifier.fit(X_train_a, y_train_a)
y_pred_nb = classifier.predict(X_test_a)
scores = accuracy_score(y_test_a, y_pred_nb)
print(scores)

In [ ]:
y_pred_test = classifier.predict(X_test_a)
y_pred=[]
for i in y_pred_test:
    if(i>=0.5):
        y_pred.append(1)
    else:
        y_pred.append(0)
cm = confusion_matrix(y_test_a,y_pred)
ax = sns.heatmap(cm, annot=True, xticklabels=["Pozitif",'Negatif'], yticklabels=['Pozitif','Negatif'],
                cbar=False,cmap='Blues')
ax.set_xlabel('Prediction')
ax.set_ylabel('Actual')
plt.show()

In [ ]:
print(classification_report(y_test_a, y_pred))

In [ ]:
# Tüm veri seti - Random Forest
rf = RandomForestClassifier(n_estimators=100, max_depth = 3, random_state=32)
rf.fit(X_train_a, y_train_a)
predictions = rf.predict(X_test_a)
score = round(accuracy_score(y_test_a, predictions), 3)
score

In [ ]:
y_pred_test = rf.predict(X_test_a)
y_pred=[]
for i in y_pred_test:
    if(i>=0.5):
        y_pred.append(1)
    else:
        y_pred.append(0)
cm = confusion_matrix(y_test_a,y_pred)
ax = sns.heatmap(cm, annot=True, xticklabels=["Pozitif",'Negatif'], yticklabels=['Pozitif','Negatif'],
                cbar=False,cmap='Blues')
ax.set_xlabel('Prediction')
ax.set_ylabel('Actual')
plt.show()

In [ ]:
print(classification_report(y_test_a, y_pred))